In [14]:
!pip install torch
!pip install matplotlib
!pip install torchvision

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
# hyperparameters
input_size = 28 # image size
num_classes = 10 # type of lables
num_epochs = 3
batch_size = 60

train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,    # 灰度图，特征图个数
                out_channels=16,  # 输出特征图, 用16个卷积核进行特征提取
                kernel_size=5,    # 卷积核大小 5x5
                stride=1,         # 步长
                padding=2         # 补零， +2圈0 if we need the size keep the same after conv, we need to set padding=(kernel_size - 1)/2
                                  #, if stride=1 
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(      # input 16, 14, 14
            nn.Conv2d(16, 32, 5, 1, 2), # output 32, 14, 14
            nn.ReLU(),
            nn.MaxPool2d(2)              # output 32, 7, 7
        )
        self.out = nn.Linear(32 * 7 * 7, 10)  # fully connected layer/dense layer 全连接层
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output
        

In [21]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [22]:
net = CNN()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001) # 定义优化器，普通随机梯度下降算法

In [24]:
#  训练模型
for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
                    
            # accuracy calculate
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
                
            print('epoch: {}[{}/{} ({:.0f}%)\t Loss: {:.6f}\t training set accuracy: {:.2f}%\t test set accuracy: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]))


epoch: 0[0/60000 (0%)	 Loss: 0.062379	 training set accuracy: 98.33%	 test set accuracy: 98.67%
epoch: 0[6000/60000 (10%)	 Loss: 0.060384	 training set accuracy: 99.06%	 test set accuracy: 98.85%
epoch: 0[12000/60000 (20%)	 Loss: 0.085340	 training set accuracy: 99.06%	 test set accuracy: 98.81%
epoch: 0[18000/60000 (30%)	 Loss: 0.002970	 training set accuracy: 99.06%	 test set accuracy: 98.92%
epoch: 0[24000/60000 (40%)	 Loss: 0.021612	 training set accuracy: 99.03%	 test set accuracy: 98.98%
epoch: 0[30000/60000 (50%)	 Loss: 0.009228	 training set accuracy: 99.02%	 test set accuracy: 99.01%
epoch: 0[36000/60000 (60%)	 Loss: 0.093205	 training set accuracy: 99.01%	 test set accuracy: 98.81%
epoch: 0[42000/60000 (70%)	 Loss: 0.051543	 training set accuracy: 98.99%	 test set accuracy: 98.98%
epoch: 0[48000/60000 (80%)	 Loss: 0.005307	 training set accuracy: 99.00%	 test set accuracy: 99.12%
epoch: 0[54000/60000 (90%)	 Loss: 0.060770	 training set accuracy: 99.01%	 test set accuracy: 98.